In [ ]:
from ctapipe.io import EventSource
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
imaging_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/imaging.lis'
simtel_file = '/Users/vdk/Software/my_code/python/jupyter_notebooks/active_jupyter_notebooks/psf/data/test_2.simtel.gz'

In [ ]:
source = EventSource(simtel_file)

event_iterator = iter(source)

In [ ]:
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
)
df

In [ ]:
df = pd.read_csv(
    imaging_file,
    sep='\s+',          # Use any whitespace as separator
    comment='#',        # Skip lines starting with '#'
    header=None,        # No header row in the data
    usecols=[4, 5],     # Columns 3 and 4 (0-based indexing: 2 and 3)
    names=['X_cm', 'Y_cm']  # Assign meaningful names
)

# Display the first few rows
print(df.head())

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(df['X_cm'], df['Y_cm'], s=1, alpha=0.5)
plt.title('Photon X vs Y Positions')
plt.xlabel('X Position (cm)')
plt.ylabel('Y Position (cm)')
plt.grid(True)
plt.show()

In [ ]:
sim_command = ('sbatch -p short -o /dev/null -e /dev/null ' +
               ' --wrap "/fefs/aswg/workspace/mykhailo.dalchenko/corsika_simtel/corsika7.7_simtelarray_2020-06-28/sim_telarray/bin/sim_telarray ' +
               '-I/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config ' +
               '-c /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config/array.cfg ' +
               '-DNUM_TELESCOPES=1 -DNO_STEREO_TRIGGER=1 -Dpower_law=2.68 ' +
               '-C telescope_altitude=__alt__ -C telescope_azimuth=__az__ -C altitude=2158 ' +
               '-C maximum_events=200, -C pedestal_events=0 ' +
               '-C random_state=auto -C ATMOSPHERIC_TRANSMISSION=atm_trans_2158_1_3_2_0_0_0.1_0.1.dat ' +
               '-C maximum_telescopes=1 -C stars=__stars__ -C show=all ' +
               '-o __output__ ' +
               '/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/dummy100000.corsika.gz"')

subrun = 0
points_per_subrun = subrun_duration * sampling_frequency
prefix = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/run7761_simulated_mispointing_stars_2.4arcsec_per_degree'
for i, (index, row) in enumerate(pointing_df.iterrows()):
    #print(f'Time: {index}, pointing Alt: {row[("Pointing", "Alt")]}, Az: {row[("Pointing", "Az")]}')
    if i%points_per_subrun == 0:
        subrun = subrun + 1
    star_filename = f'{prefix}/subrun_{subrun}/stars/{index}.dat'
    outputdir = f'{prefix}/subrun_{subrun}/output/'
    os.makedirs(os.path.dirname(star_filename), exist_ok=True)
    os.makedirs(os.path.dirname(outputdir), exist_ok=True)
    with open(star_filename, 'w') as f:
        for star_label in stars.keys():
            f.write(f'{row[(star_label, "Az")]} {row[(star_label, "Alt")]} {star_label}\n')
    cmd = sim_command.replace('__alt__', str(row[("Pointing", "Alt")]))
    cmd = cmd.replace('__az__', str(row[("Pointing", "Az")]))
    cmd = cmd.replace('__stars__', star_filename)
    cmd = cmd.replace('__output__', f'{outputdir}/{index}.simtel.gz')
    #logger.debug('simulation cmd\n %s', cmd) 
    subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
sim_command = ('sbatch -p short -o /dev/null -e /dev/null ' +
               ' --wrap "/fefs/aswg/workspace/mykhailo.dalchenko/corsika_simtel/corsika7.7_simtelarray_2020-06-28/sim_telarray/bin/sim_telarray ' +
               '-I/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config ' +
               '-c /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config/array.cfg ' +
               '-DNUM_TELESCOPES=1 -DNO_STEREO_TRIGGER=1 -Dpower_law=2.68 ' +
               '-C telescope_altitude=80 -C telescope_azimuth=180 -C altitude=2158 ' +
               '-C maximum_events=200, -C pedestal_events=0 ' +
               '-C random_state=auto -C ATMOSPHERIC_TRANSMISSION=atm_trans_2158_1_3_2_0_0_0.1_0.1.dat ' +
               '-C maximum_telescopes=1 -C stars=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/stars/10zenith_star.dat -C show=all ' +
               'IMAGING_LIST imaging.lis ' +
               '-o /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/zenith20/test.simtel.gz ' +
               '/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/dummy100000.corsika.gz ')

subrun = 0
points_per_subrun = subrun_duration * sampling_frequency
prefix = '/fefs/aswg/workspace/mykhailo.dalchenko/pointing/simulation/run7761_simulated_mispointing_stars_2.4arcsec_per_degree'
for i, (index, row) in enumerate(pointing_df.iterrows()):
    #print(f'Time: {index}, pointing Alt: {row[("Pointing", "Alt")]}, Az: {row[("Pointing", "Az")]}')
    if i%points_per_subrun == 0:
        subrun = subrun + 1
    star_filename = f'{prefix}/subrun_{subrun}/stars/{index}.dat'
    outputdir = f'{prefix}/subrun_{subrun}/output/'
    os.makedirs(os.path.dirname(star_filename), exist_ok=True)
    os.makedirs(os.path.dirname(outputdir), exist_ok=True)
    with open(star_filename, 'w') as f:
        for star_label in stars.keys():
            f.write(f'{row[(star_label, "Az")]} {row[(star_label, "Alt")]} {star_label}\n')
    cmd = sim_command.replace('__alt__', str(row[("Pointing", "Alt")]))
    cmd = cmd.replace('__az__', str(row[("Pointing", "Az")]))
    cmd = cmd.replace('__stars__', star_filename)
    cmd = cmd.replace('__output__', f'{outputdir}/{index}.simtel.gz')
    #logger.debug('simulation cmd\n %s', cmd) 
    subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
import os

sim_command = ('sbatch -p short -o /dev/null -e /dev/null ' +
               ' --wrap "/fefs/aswg/workspace/mykhailo.dalchenko/corsika_simtel/corsika7.7_simtelarray_2020-06-28/sim_telarray/bin/sim_telarray ' +
               '-I/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config ' +
               '-c /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/lst-sim-config/array.cfg ' +
               '-DNUM_TELESCOPES=1 -DNO_STEREO_TRIGGER=1 -Dpower_law=2.68 ' +
               '-C telescope_altitude=80 -C telescope_azimuth=180 -C altitude=2158 ' +
               '-C maximum_events=200, -C pedestal_events=0 ' +
               '-C random_state=auto -C ATMOSPHERIC_TRANSMISSION=atm_trans_2158_1_3_2_0_0_0.1_0.1.dat ' +
               '-C maximum_telescopes=1 -C stars=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/stars/10zenith_star.dat -C show=all ' +
               'IMAGING_LIST imaging.lis ' +
               '-o /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/zenith10/test.simtel.gz ' +
               '/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/dummy100000.corsika.gz ')

job_name = 'star10deg'
job_file = open(f"/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/{job_name}.job", "w")
job_file.writelines("#!/bin/bash\n")
job_file.writelines("#BATCH --partition=short\n")
job_file.writelines("#SBATCH --time=1:59:00\n")
job_file.writelines("#SBATCH --ntasks=1\n")
job_file.writelines("#SBATCH --mem-per-cpu=6000 # in MB\n")
job_file.writelines(f"#SBATCH --output=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/log/{job_name}.log\n")
job_file.writelines(f"#SBATCH --error=/fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/error/{job_name}.err\n")
job_file.writelines("\n")
job_file.writelines(sim_command)
job_file.close()
command = f"sbatch /fefs/aswg/workspace/vadym.voitsekhovskyi/psf/jobs/{job_name}.job"
os.system(command)
print(f"Sbatch name {job_name} runned")


In [ ]:
type(sim_command)